In [1]:
import json
import time

In [2]:
context_corpus = json.load(open('dataset/paragraph_context.json', 'r', encoding='utf8')) 
question_corpus = json.load(open('dataset/question_context.json', 'r', encoding='utf8'))
train_labels_json = json.load(open('dataset/train_labels.json', 'r', encoding='utf8'))
test_labels_json = json.load(open('dataset/test_labels.json', 'r', encoding='utf8'))

In [3]:
from rank_bm25 import BM25Okapi

In [4]:
test_document = [context_corpus[t] for t in test_labels_json]
test_question = [question_corpus[t] for key in test_labels_json.keys() for t in test_labels_json[key]]

In [5]:
from konlpy.tag import Okt, Komoran

In [6]:
okt = Okt()

In [8]:
st = time.time()
bm25 = BM25Okapi([okt.morphs(doc) for doc in test_document])
print(time.time() - st)

935.4163348674774


In [9]:
start_time = time.time()
answer = []
for q in [okt.morphs(question) for question in test_question]:
    answer.append(bm25.get_top_n(q, test_document, n=150))

end_time = time.time()

In [12]:
end_time - start_time

8345.675825357437

In [13]:
test_question_ids = [t for key in test_labels_json.keys() for t in test_labels_json[key]]

In [14]:
test_document_ids = {context_corpus[t]:t for t in test_labels_json}

In [15]:
bm25_space_result = {}

In [16]:
for cnt, ans in enumerate(answer):
    bm25_space_result[test_question_ids[cnt]] = []
    for a in ans:
        bm25_space_result[test_question_ids[cnt]].append(test_document_ids[a])

In [17]:
test_label_clean = {}
for test_label in test_labels_json.keys():
    for test in test_labels_json[test_label]:
        test_label_clean[test] = test_label

In [18]:
mrr = 0

for clean in test_label_clean.keys():
    try:
        mrr += 1 / (bm25_space_result[clean].index(test_label_clean[clean]) + 1)
    except:
        pass
    
print(mrr / len(test_label_clean))

0.8734029146886991


In [20]:
with open('bm25_okt_result.json', 'w') as f:
    json.dump(bm25_space_result, f)